In [1]:
#1. rutin1 import module
import pandas as pd
import os
import sys
import warnings
import matplotlib.pyplot as plt
#from sklearn.metrics.pairwise import cosine_similarity

warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
#2. rutin2 membuat syspath ke root utk aktifkan __init__.py
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(repo_root)

In [3]:
#3. rutin3 Load the dataset
#dataset_path = os.path.join(repo_root, "notebooks/postager_nlp-id/dataset_ekstraksi_r29_pos_sm.xlsx")
dataset_path = os.path.join(repo_root, "data/dataset_ekstraksi_r29_lg.xlsx")
df = pd.read_excel(dataset_path)
df["text"] = df["judul"] +". "+ df["isi"]

In [4]:
# tuning paramater
tuning_multiplier = 1  #aktor pengali dari score jika kata tersebut merupakan frase. default = 1 (perlu variasi 0.6 - 0.75)
tuning_f_phrase = 3  #score minimum utk bisa disebut frase
m_prediksi = 10  #jumlah top -n keyword prediksi
n_top_phrase = 3   #jumlah frase yg akan di cari dalam fungsi get_top_phrase

In [5]:
# Preprocess
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemover, ArrayDictionary

def preprocess(text):
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = text.replace('.', '. ')
    text = re.sub('[^a-zA-Z.]', ' ', text)
    text = text.lower()
    text = re.sub("(\\d|\\W)+"," ",text)
    text = text.strip()

    #stopwords_path = os.path.join(repo_root, "data/all_stop_words.txt")
    stopwords_path = os.path.join(repo_root, "notebooks/stopwords_tuning/all_stop_words.txt")
    with open(stopwords_path, 'r') as f:
        stopwords = [line.strip() for line in f]

    dictionary = ArrayDictionary(stopwords)
    str = StopWordRemover(dictionary)
    text = str.remove(text)

    return text

df['preprocessed_text'] = df['text'].apply(preprocess)
df_tr = df['preprocessed_text'].apply(preprocess)

In [6]:
import numpy as np
import math
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import CountVectorizer

def build_graph(vocab_len, processed_text, vocabulary):
    """
    Builds a weighted edge graph based on co-occurrences of words in the text.
    + perlu ada tambahan formula untuk menghitung score kata yg ada dalam title menjadi lebih besar. (1, 1.5, 2)
    """
    weighted_edge = np.zeros((vocab_len, vocab_len), dtype=np.float32)
    score = np.ones((vocab_len), dtype=np.float32)
    window_size = 3  
    covered_coocurrences = []

    for i in range(vocab_len):
        for j in range(vocab_len):
            if j == i:
                weighted_edge[i][j] = 0
            else:
                for window_start in range(len(processed_text) - window_size):
                    window_end = window_start + window_size
                    window = processed_text[window_start:window_end]
                    if (vocabulary[i] in window) and (vocabulary[j] in window):
                        index_of_i = window_start + window.index(vocabulary[i])
                        index_of_j = window_start + window.index(vocabulary[j])
                        if [index_of_i,index_of_j] not in covered_coocurrences:
                            weighted_edge[i][j] += 1 / math.fabs(index_of_i - index_of_j)
                            covered_coocurrences.append([index_of_i, index_of_j])

    inout = np.sum(weighted_edge, axis=1)
  
    MAX_ITERATIONS = 50
    d = 0.85
    threshold = 0.0001
    for _ in range(MAX_ITERATIONS):
        prev_score = np.copy(score)
        for i in range(vocab_len):
            summation = 0
            for j in range(vocab_len):
                if weighted_edge[i][j] != 0:
                    summation += (weighted_edge[i][j] / inout[j]) * score[j]
            score[i] = (1 - d) + d * summation
        if np.sum(np.fabs(prev_score - score)) <= threshold:
            break

    return vocabulary, score

def score_phrases(unique_phrases, vocabulary, score, multiplier=tuning_multiplier):
    """
    Computes the score of each phrase using the given vocabulary, word scores, and multiplier.
    """
    phrase_scores = []
    keywords = []
    for phrase in unique_phrases:
        phrase_score = 0
        keyword = ''
        for word in phrase:
            keyword += str(word) + " "
            phrase_score += score[vocabulary.index(word)]
        phrase_score *= multiplier
        phrase_scores.append(phrase_score)
        keywords.append(keyword.strip())

    return keywords, phrase_scores


def get_top_phrase(corpus, n=n_top_phrase):  #perlu ada improvement karena phrase yg di hasilkan masih blm proper
    vec1 = CountVectorizer(ngram_range=(2,3),  
            max_features=2000).fit([corpus])
    bag_of_words = vec1.transform([corpus])
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    # perlu di buat filter jika pola tidak mengikuti kaidah kata majemuk indonesia di excludekan.
    return words_freq[:n]

def predict_keywords(text, m=10, f_phrase=5, tuning_multiplier=1):
    """
    Predicts the top m keywords and top f_phrase phrases for the given text.
    processed_text = text keseluruhan
    vocabulary = unique word dalam proccesesed_text
    """
    processed_text = word_tokenize(text)
    vocabulary = list(set(processed_text))
    vocab_len = len(vocabulary)
    vocabulary, score = build_graph(vocab_len, processed_text, vocabulary)
    unigram = pd.DataFrame({
        'Keyword': vocabulary,
        'Score': score
    }).nlargest(m, 'Score')
    
    bi_trigram = pd.DataFrame(get_top_phrase(text, n=50), columns=['Phrase', 'Score'])
    bi_trigram = bi_trigram[bi_trigram['Score'] >= f_phrase]
    bi_trigram['Tokens'] = bi_trigram['Phrase'].apply(word_tokenize)
    unique_phrases = bi_trigram['Tokens'].values.tolist()
    keywords, phrase_scores = score_phrases(unique_phrases, vocabulary, score, tuning_multiplier) #BUG_1 not accesed by pylance, krn tidak di gunakan di procss selanjutnya
    # memasukan score ke dalam dataframe
    bi_trigram = pd.DataFrame({
        'Phrase': keywords,
        'Score': phrase_scores
    }).nlargest(m, 'Score')

      # Combine unigram and bi_trigram dataframes
    predict_keywords = pd.concat([unigram, bi_trigram[['Phrase', 'Score']].rename(columns={'Phrase': 'Keyword'})])\
                    .sort_values('Score', ascending=False)\
                    .nlargest(m, 'Score')\
                    .reset_index(drop=True)

    return predict_keywords

In [7]:
# Predict keywords for all sentences in the dataframe and save the in dataframe
#from utils import build_graph, score_phrases, get_top_phrase, predict_keywords, write_excel
predict_textrank = pd.DataFrame()
#for i in df_tr.index:
for i in df.loc[490:].index:
    print('Processing index', i, end='...! ')
    keyphrase = predict_keywords(df_tr[i], m_prediksi, tuning_f_phrase, tuning_multiplier).reset_index(drop=True)
    a = pd.DataFrame(keyphrase.Keyword).T.reset_index(drop=True)
    b = pd.DataFrame(keyphrase.Score).round(2).T.reset_index(drop=True)
    keyphrase = pd.concat([a, b], axis=1)
    
    # Ensure that keyphrase has the same number of columns as max_columns by filling in with NaN
    max_columns = 20
    missing_cols = max_columns - keyphrase.shape[1]
    if missing_cols > 0:
        nan_cols = pd.DataFrame(np.nan, index=keyphrase.index, columns=[f'col{col}' for col in range(keyphrase.shape[1], max_columns)])
        keyphrase = pd.concat([keyphrase, nan_cols], axis=1)

    predict_textrank = pd.concat([predict_textrank, keyphrase], ignore_index=True)
    print('Done')
#predict_textrank.columns = ['key_1', 'key_2','key_3','score_1', 'score_2','score_3'] 
predict_textrank.columns = ['key_1', 'key_2','key_3', 'key_4', 'key_5','key_6', 'key_7', 'key_8','key_9','key_10','score_1', 'score_2','score_3','score_4', 'score_5','score_6','score_7', 'score_8','score_9','score_10'] 

Processing index 490...! Done
Processing index 491...! Done
Processing index 492...! Done
Processing index 493...! Done
Processing index 494...! Done
Processing index 495...! Done
Processing index 496...! Done
Processing index 497...! Done
Processing index 498...! Done
Processing index 499...! Done
Processing index 500...! Done
Processing index 501...! Done
Processing index 502...! Done
Processing index 503...! Done
Processing index 504...! Done
Processing index 505...! Done
Processing index 506...! Done
Processing index 507...! Done
Processing index 508...! Done
Processing index 509...! Done
Processing index 510...! Done
Processing index 511...! Done
Processing index 512...! Done
Processing index 513...! Done
Processing index 514...! Done
Processing index 515...! Done
Processing index 516...! Done
Processing index 517...! Done
Processing index 518...! Done
Processing index 519...! Done
Processing index 520...! Done
Processing index 521...! Done
Processing index 522...! Done
Processing

Traceback (most recent call last):
  File "/Users/jim/anaconda3/envs/kw_ina/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/68/k0y245f57539r_hm0r4dpqtw0000gn/T/ipykernel_50300/95411966.py", line 7, in <module>
    keyphrase = predict_keywords(df_tr[i], m_prediksi, tuning_f_phrase, tuning_multiplier).reset_index(drop=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/68/k0y245f57539r_hm0r4dpqtw0000gn/T/ipykernel_50300/160117697.py", line 89, in predict_keywords
    vocabulary, score = build_graph(vocab_len, processed_text, vocabulary)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/68/k0y245f57539r_hm0r4dpqtw0000gn/T/ipykernel_50300/160117697.py", line -1, in build_graph
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Trac

EVALUATION

In [ ]:
from utils import eval

targets = df[["k1", "k2", "k3","k4", "k5", "k6","k7"]].values.tolist()
df_targets = pd.DataFrame(targets)

In [ ]:
# Evaluation TextRank top 10
predict_textrank_list_10 = predict_textrank[['key_1','key_2','key_3', 'key_4','key_5','key_6', 'key_7','key_8','key_9', 'key_10']].values.tolist()
eval_textrank_10 = eval(predict_textrank_list_10, targets, True).round(3)
eval_textrank_10.columns = ['key_1','key_2','key_3', 'key_4','key_5','key_6', 'key_7','key_8','key_9', 'key_10','strict_recall', 'strict_prec', 'flex_recall','flex_prec']
eval_textrank_10 = eval_textrank_10[['key_1','key_2','key_3', 'key_4','key_5','key_6', 'key_7','key_8','key_9', 'key_10', 'flex_recall','flex_prec']] # untuk menyederhanakan hasil evaluasi
eval_textrank_10.head(3)

,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,flex_recall,flex_prec
0,no_match,no_match,no_match,no_match,no_match,no_match,no_match,no_match,no_match,no_match,0.000,0.0
1,partial_match,partial_match,full_match,full_match,no_match,no_match,no_match,no_match,no_match,no_match,0.571,0.4
2,no_match,partial_match,no_match,no_match,no_match,no_match,no_match,partial_match,no_match,no_match,0.286,0.2


In [ ]:
# Calculate TextRank Score, using flexible score : exact maatch =1, partial match = 1, no match = 0
textrank_recall_10 = eval_textrank_10['flex_recall'].mean()
textrank_prec_10 = eval_textrank_10['flex_prec'].mean()
textrank_f1_10 = 2 * (textrank_prec_10 * textrank_recall_10) / (textrank_prec_10 + textrank_recall_10)

# Create a DataFrame with the scores
summary_10 = pd.DataFrame({'textrank': [textrank_recall_10, textrank_prec_10, textrank_f1_10]}, index=['recall', 'precision', 'F1'])
summary_10 = summary_10.round(3)
summary_10

,textrank
recall,0.320
precision,0.224
F1,0.264


In [ ]:
# Evaluation TextRank top 5
predict_textrank_list_5 = predict_textrank[['key_1','key_2','key_3', 'key_4','key_5']].values.tolist()
eval_textrank_5 = eval(predict_textrank_list_5, targets, True).round(3)
eval_textrank_5.columns = ['key_1','key_2','key_3', 'key_4','key_5','strict_recall', 'strict_prec', 'flex_recall','flex_prec']
eval_textrank_5 = eval_textrank_5[['key_1','key_2','key_3', 'key_4','key_5', 'flex_recall','flex_prec']] # untuk menyederhanakan hasil evaluasi
eval_textrank_5.head(3)

,key_1,key_2,key_3,key_4,key_5,flex_recall,flex_prec
0,no_match,no_match,no_match,no_match,no_match,0.000,0.0
1,partial_match,partial_match,full_match,full_match,no_match,0.571,0.8
2,no_match,partial_match,no_match,no_match,no_match,0.143,0.2


In [ ]:
# Calculate TextRank Score, using flexible score : exact maatch =1, partial match = 1, no match = 0
textrank_recall_5 = eval_textrank_5['flex_recall'].mean()
textrank_prec_5 = eval_textrank_5['flex_prec'].mean()
textrank_f1_5 = 2 * (textrank_prec_5 * textrank_recall_5) / (textrank_prec_5 + textrank_recall_5)

# Create a DataFrame with the scores
summary_5 = pd.DataFrame({'textrank': [textrank_recall_5, textrank_prec_5, textrank_f1_5]}, index=['recall', 'precision', 'F1'])
summary_5 = summary_5.round(3)
summary_5

,textrank
recall,0.192
precision,0.269
F1,0.224


In [ ]:
# Evaluation TextRank top 3
predict_textrank_list_3 = predict_textrank[['key_1','key_2','key_3']].values.tolist()
eval_textrank_3 = eval(predict_textrank_list_3, targets, True).round(3)
eval_textrank_3.columns = ['key_1', 'key_2','key_3','strict_recall', 'strict_prec', 'flex_recall','flex_prec']
eval_textrank_3 = eval_textrank_3[['key_1', 'key_2','key_3', 'flex_recall','flex_prec']] # untuk menyederhanakan hasil evaluasi
eval_textrank_3.head(3)

,key_1,key_2,key_3,flex_recall,flex_prec
0,no_match,no_match,no_match,0.000,0.000
1,partial_match,partial_match,full_match,0.429,1.000
2,no_match,partial_match,no_match,0.143,0.333


In [ ]:
# Calculate TextRank Score, using flexible score : exact maatch =1, partial match = 1, no match = 0
textrank_recall_3 = eval_textrank_3['flex_recall'].mean()
textrank_prec_3 = eval_textrank_3['flex_prec'].mean()
textrank_f1_3 = 2 * (textrank_prec_3 * textrank_recall_3) / (textrank_prec_3 + textrank_recall_3)

# Create a DataFrame with the scores
summary_3 = pd.DataFrame({'textrank': [textrank_recall_3, textrank_prec_3, textrank_f1_3]}, index=['recall', 'precision', 'F1'])
summary_3 = summary_3.round(3)
summary_3

,textrank
recall,0.108
precision,0.253
F1,0.152


In [ ]:
# Combine dataframe predict_textrank, df_targets and eval_textrank
predict_textrank_10 = pd.concat([predict_textrank, df_targets, eval_textrank_10], axis=1)
predict_textrank_10.head(3)

,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,...,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,flex_recall,flex_prec
0,personil diajukan fase,diajukan fase tender,organisasi diajukan fase,personil pengganti,personil diajukan,fase tender,organisasi diajukan,personil,diajukan fase,tender,...,no_match,no_match,no_match,no_match,no_match,no_match,no_match,no_match,0.000,0.0
1,document,template,pengelolaan,acuan,demikian,facilities,processing,biru,atas,tiung,...,full_match,full_match,no_match,no_match,no_match,no_match,no_match,no_match,0.571,0.4
2,ruangan kantor,kantor,ruangan,artikel,appendix,usulan,scope,company,dengan,komposisi,...,no_match,no_match,no_match,no_match,no_match,partial_match,no_match,no_match,0.286,0.2


In [ ]:
# Combine dataframe predict_textrank, df_targets and eval_textrank
predict_textrank_5 = pd.concat([predict_textrank, df_targets, eval_textrank_5], axis=1)
predict_textrank_5.head(3)

,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,...,4,5,6,key_1,key_2,key_3,key_4,key_5,flex_recall,flex_prec
0,personil diajukan fase,diajukan fase tender,organisasi diajukan fase,personil pengganti,personil diajukan,fase tender,organisasi diajukan,personil,diajukan fase,tender,...,NaN,NaN,NaN,no_match,no_match,no_match,no_match,no_match,0.000,0.0
1,document,template,pengelolaan,acuan,demikian,facilities,processing,biru,atas,tiung,...,dokumen,NaN,NaN,partial_match,partial_match,full_match,full_match,no_match,0.571,0.8
2,ruangan kantor,kantor,ruangan,artikel,appendix,usulan,scope,company,dengan,komposisi,...,services for company,exhibit a,NaN,no_match,partial_match,no_match,no_match,no_match,0.143,0.2


In [ ]:
# Combine dataframe predict_textrank, df_targets and eval_textrank
predict_textrank_3 = pd.concat([predict_textrank, df_targets, eval_textrank_3], axis=1)
predict_textrank_3.head(3)

,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,...,2,3,4,5,6,key_1,key_2,key_3,flex_recall,flex_prec
0,personil diajukan fase,diajukan fase tender,organisasi diajukan fase,personil pengganti,personil diajukan,fase tender,organisasi diajukan,personil,diajukan fase,tender,...,usulan,pengganti,NaN,NaN,NaN,no_match,no_match,no_match,0.000,0.000
1,document,template,pengelolaan,acuan,demikian,facilities,processing,biru,atas,tiung,...,acuan,pengelolaan,dokumen,NaN,NaN,partial_match,partial_match,full_match,0.429,1.000
2,ruangan kantor,kantor,ruangan,artikel,appendix,usulan,scope,company,dengan,komposisi,...,lingkup kerja,akomodasi,services for company,exhibit a,NaN,no_match,partial_match,no_match,0.143,0.333


In [ ]:
# Write predictions to excel file
from utils import write_excel

sheet_name_10 = '1_tr_phrase_countvectorizer'
sheet_name_5 = '1_tr_phrase_countvectorizer'
sheet_name_3 = '1_tr_phrase_countvectorizer'

output_file = '1_tr_phrase_countvectorizer.xlsx'
write_excel(predict_textrank_10, sheet_name_10, output_file)
write_excel(predict_textrank_5, sheet_name_5, output_file)
write_excel(predict_textrank_3, sheet_name_3, output_file)

/Users/jim/Documents/GitHub/kw_ina_extraction/utils/ia_file_operation.py:16: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
/Users/jim/Documents/GitHub/kw_ina_extraction/utils/ia_file_operation.py:25: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
